In [2]:
model.model.criterion

In [133]:
import torch

batch = {
    'img': torch.randn(4, 3, 640, 640),
    'cls': torch.tensor([[22.],
        [22.],
        [45.],
        [45.],
        [50.],
        [45.],
        [49.],
        [49.],
        [49.],
        [49.],
        [45.],
        [45.],
        [58.],
        [75.],
        [23.],
        [23.],
        [45.],
        [45.],
        [49.],
        [49.],
        [49.],
        [49.]]),
    'bboxes': torch.tensor([[0.6774, 0.2597, 0.6453, 0.5194],
        [0.6774, 0.7985, 0.6453, 0.4030],
        [0.3255, 0.7542, 0.6510, 0.4917],
        [0.4058, 0.3885, 0.5542, 0.3969],
        [0.2953, 0.7902, 0.5489, 0.4195],
        [0.1709, 0.5315, 0.3418, 0.6511],
        [0.3062, 0.2930, 0.1311, 0.0808],
        [0.4465, 0.2907, 0.1008, 0.0810],
        [0.3300, 0.3716, 0.1458, 0.1224],
        [0.3017, 0.2485, 0.1645, 0.1234],
        [0.8502, 0.7542, 0.2996, 0.4917],
        [0.8492, 0.5315, 0.3015, 0.6511],
        [0.0622, 0.3344, 0.1243, 0.6688],
        [0.0271, 0.4510, 0.0543, 0.4181],
        [0.6824, 0.3298, 0.3706, 0.5122],
        [0.2239, 0.6323, 0.2276, 0.0951],
        [0.7197, 0.8560, 0.5504, 0.2881],
        [0.8916, 0.8639, 0.2167, 0.2722],
        [0.8186, 0.8142, 0.1302, 0.0802],
        [0.6793, 0.8119, 0.1001, 0.0804],
        [0.7950, 0.8923, 0.1448, 0.1215],
        [0.8230, 0.7701, 0.1633, 0.1225]]),
    'batch_idx': torch.tensor([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3.])
}


In [162]:
z = model.model(batch)

AttributeError: 'DetectionModel' object has no attribute 'args'

In [ ]:
model.argsw

AttributeError: 'DetectionModel' object has no attribute 'args'

In [2]:
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel

# from ultralytics.ultralytics.nn.tasks import DetectionModel

# Load a model
model = YOLO("yolo11n.pt")
model.model = DetectionModel("/Users/apple/YOLO/ultralytics/ultralytics/cfg/models/11/yolo11.yaml", nc=4, verbose=False )
model.model.args = model.args
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)


Overriding model.yaml nc=80 with nc=4
WARNING ⚠️ no model scale passed. Assuming scale='n'.


AttributeError: 'DetectionModel' object has no attribute 'args'

In [3]:
model.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

model.model.criterion = model.model.init_criterion()

In [ ]:
model.model.criterion.hyp







































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































nn


{'task': 'detect',
 'data': '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml',
 'imgsz': 640,
 'single_cls': False,
 'model': 'yolo11n.pt'}

In [ ]:
            self.criterion = self.init_criterion()


In [132]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [ ]:
# results = model(["/Users/apple/Downloads/HienDang.png"])  # return a list of Results objects



0: 576x640 1 person, 80.4ms
Speed: 3.8ms preprocess, 80.4ms inference, 1.2ms postprocess per image at shape (1, 3, 576, 640)


In [6]:
type(model)

ultralytics.models.yolo.model.YOLO

In [16]:
# model.model.loss

In [86]:
# import inspect



# # Get the source code of the function
# source_code = inspect.getsource(model.model._predict_once)
# print(source_code)


In [42]:
from transformers import AutoImageProcessor, AutoModel
import torch
from PIL import Image

image = Image.open("/Users/apple/Downloads/HienDang.png").convert("RGB")

image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
resnet = AutoModel.from_pretrained("microsoft/resnet-18")

inputs = image_processor([image], return_tensors="pt")

with torch.no_grad():
    logits = resnet(**inputs, output_hidden_states=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [43]:
for e in logits.hidden_states:
    print(e.shape)

torch.Size([1, 64, 56, 56])
torch.Size([1, 64, 56, 56])
torch.Size([1, 128, 28, 28])
torch.Size([1, 256, 14, 14])
torch.Size([1, 512, 7, 7])


In [80]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [125]:
import torch
from dataclasses import dataclass

@dataclass
class Share_data:
    x1: torch.Tensor
    x2: torch.Tensor
    x3: torch.Tensor


class B_block(torch.nn.Module):
    def __init__(self, i, backbone, idx, data):
        super(B_block, self).__init__()
        self.backbone = backbone
        self.idx = idx
        self.data = data
        self.i = i
        self.f = -1
        hidden_sizes = backbone.config.hidden_sizes
        self.proj_1 = torch.nn.Conv2d(hidden_sizes[-3], 128, 1)
        self.proj_2 = torch.nn.Conv2d(hidden_sizes[-2], 128, 1)
        self.proj_3 = torch.nn.Conv2d(hidden_sizes[-1], 256, 1)

    def forward(self, x):
        if self.idx == 1:
            hidden_states = self.backbone(x, output_hidden_states=True).hidden_states
            x1, x2, x3 = hidden_states[-3:]
            x1 = self.proj_1(x1)
            x2 = self.proj_2(x2)
            x3 = self.proj_3(x3)
            self.data.x1 = x1
            self.data.x2 = x2
            self.data.x3 = x3
            return x1
        elif self.idx == 2:
            return self.data.x2
        elif self.idx == 3:
            return self.data.x3


class E_block(torch.nn.Module):
    def __init__(self, i):
        super(E_block, self).__init__()
        self.f = -1
        self.i = i

    def forward(self, x):
        return x


sdata = Share_data(None, None, None)
model.model.model[0] = E_block(0)
model.model.model[1] = E_block(1)
model.model.model[2] = E_block(2)
model.model.model[3] = E_block(3)
model.model.model[4] = B_block(4, resnet, 1, sdata)
model.model.model[5] = E_block(5)
model.model.model[6] = B_block(6, resnet, 2, sdata)
model.model.model[7] = E_block(7)
model.model.model[8] = E_block(8)
model.model.model[9] = E_block(9)
model.model.model[10] = B_block(10, resnet, 3, sdata)

In [117]:

# x = torch.randn(1, 3, 224, 224)
# x1 = b1(x)
# x2 = b2(x1)
# x3 = b3(x2)

# print(x1.shape)
# print(x2.shape)
# print(x3.shape)

In [119]:
model.model.model[0].i

0

In [121]:
x = torch.randn(4, 3, 224, 224)
y, dt, embeddings = [], [], []  # outputs
for k, m in enumerate(model.model.model):  # run model
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

    x = m(x)  # run
    y.append(x if m.i in model.model.save else None)  # save output
      


torch.Size([4, 3, 224, 224])
torch.Size([4, 128, 28, 28]) torch.Size([4, 256, 14, 14]) torch.Size([4, 512, 7, 7])


In [15]:
model.model.save

[4, 6, 10, 13, 16, 19, 22]

In [114]:
for k, z in enumerate(y):
    if z is not None:
        print(k, z.shape)
    else:
        print(k, z)

0 None
1 None
2 None
3 None


In [74]:
for k, z in enumerate(y):
    if z is not None:
        print(k, z.shape)
    else:
        print(k, z)

0 None
1 None
2 None
3 None
4 torch.Size([4, 128, 28, 28])
5 None
6 torch.Size([4, 128, 14, 14])
7 None
8 None
9 None
10 torch.Size([4, 256, 7, 7])
11 None
12 None
13 torch.Size([4, 128, 14, 14])
14 None
15 None
16 torch.Size([4, 64, 28, 28])
17 None
18 None
19 torch.Size([4, 128, 14, 14])
20 None
21 None
22 torch.Size([4, 256, 7, 7])
23 None


In [14]:
for m in model.model.model:
    print(m)
    print('='*100)


Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (act): SiLU(inplace=True)
)
Conv(
  (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (act): SiLU(inplace=True)
)
C3k2(
  (cv1): Conv(
    (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (m): ModuleList(
    (0): Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
    )
  )
)
Conv(
  (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (act): SiLU(inplace=True)
)
C3k2(
  (cv1): Conv(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(in

In [ ]:
            self.trainer.model = self.trainer.get_model(weights=self.model if self.ckpt else None, cfg=self.model.yaml)


In [13]:
model.trainer

In [1]:
# model.model

Ultralytics 8.3.89 🚀 Python-3.10.11 torch-2.3.1 CPU (unknown)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=tor

train: Scanning /Users/apple/YOLO/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
val: Scanning /Users/apple/YOLO/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


No module named 'seaborn'
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/apple/YOLO/ultralytics/runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'shape'

In [4]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [5]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen


In [2]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [4]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 51558, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (462/462), done.
remote: Total 51558 (delta 448), reused 202 (delta 197), pack-reused 50899 (from 4)
Receiving objects: 100% (51558/51558), 29.99 MiB | 690.00 KiB/s, done.
Resolving deltas: 100% (37820/37820), done.


In [6]:
%cd ultralytics

/Users/apple/YOLO/ultralytics


/Users/apple/Library/Python/3.10/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
